# Table Of Content:
* Preliminaries
    * [Before starting the lab !](#lab0)
    * [Structure of the lab](#structure)
    * [Objective of the lab](#objective)
    * [How to work for the lab?](#howto)
    * [Setting up the scene](#scene)
* [Part 1: Geometry](#part1)
    * [I. Computing the target configurations (Inverse Geometry)](#IG)
    * [II. Motion planning](#motion_planning)
* [Part 2: Dynamics](#part2)
    * [I. From a path to a trajectory](#TO)
    * [II. Implementing a control law](#control)




# Before starting the lab ! <a class="anchor" id="lab0"></a>
Before getting into the lab, it is recommended to follow the notebook [lab_0_geometry_with_pinocchio.ipynb](./lab_0_geometry_with_pinocchio.ipynb) to get familiar with the pinocchio API.


# Structure of the lab:  <a class="anchor" id="structure"></a>
The lab is divided into separate python files, each designed for you to address a sub-problem atomically. These instructions will indicate where you should implement each task. At the end of each file, in the 'main' 
section you can locally test your functions. It is important that you **do not modify the names and signatures** of the methods provided: When marking the lab, I will in first instance run code that will use these functions to evaluate quantitatively the methods you proposed. 

# Objective of the lab <a class="anchor" id="objective"></a>
Use both effectors of the nextage robot to grab a box and bring it to a target location.
You will first plan a valid motion that brings the robot to a grasping configuration, then moves the box to 
a desired location while avoiding collisions. For this you will use a combination of motion planning and numerical optimisation.

Once this motion plan will be computed, you will test it in a dynamics simulator using a control method of your choice.
This will be the objective of part 2 of the lab.

In the optional part 3 of the lab (for exceptional marks), you will be asked to self-propose a more complex task to achieve, which will require you to implement additional features to your framework. 

# How to work for the lab? <a class="anchor" id="howto"></a>
You have been used to notebooks for the tutorials, and these instructions also take the form of a notebook. To implement this lab I personally chose to work directly with a python IDE and I recommend to do the same. I worked with spyder but any python IDE might work as well. You can decide to work using notebooks if you prefer, this is not a problem, as long as your final deliverables meet the requirement specification.

**In any case, keep your code in a versioning system**. You are free to use github gitlab or whatever service you are more comfortable working with. The easiest way to work is to "fork" the lab repository from github into your own account and adding this new repository as a remote server.



## Code production
You are free to reuse code from the tutorials or any other source as long as you explicitely **cite its origin both in the code and in your report**. You are free to use any method from the pinocchio API and to create as many methods as you would like. **If you want to use non-native python libraries**, we must discuss this. Remember that I will have to run your code to assess your lab!


## I don't like the approach you have proposed to solve the problem. Can I do my own thing?
Yes... and no. First of all, I would suggest that you discuss this with the TAs / myself before going for it. Secondly, you will see in the submission requirements that I only need some methods to be implemented for me to assess quantitatively your work. I consider that if all of these methods are implemented you followed the instructions. The report will then give you a chance to justify your approach. This should give you a lot of freedom. In particular at step 2 I propose to use motion planning to compute a reference path for the robot. If you choose a different approach it does not matter to me as long as we discussed it before and that it is not hard-coded somehow. 

# Setting up the scene <a class="anchor" id="scene"></a>

To ensure efficient use of resources and to prevent the unintentional creation of multiple Meshcat server instances, I recommend initializing the server through the command line. Here's how you can do it:

- Open a terminal. If you're on Ubuntu, you can quickly do this by pressing `ctrl + alt + t`.
- Enter the following command and press `Enter`:
   ```
   meshcat-server 
   ```
- Upon running the command, you'll receive an output that includes the "zmq_url". This is the address to which you will connect.

**Tip**: You can run terminal commands directly from the Jupyter Notebook by prefixing them with `!`. However, for the purpose of this lab, I recommend initializing the Meshcat server directly from the terminal.


In [21]:
!pip install numpy==1.26.3


A helper function has been implemented to automatically connect to meshcat server, load the scene and the robot, and setup the collision handler in pinocchio for you.

In [22]:
from tools import setupwithmeshcat
robot, cube, viz = setupwithmeshcat(url="tcp://127.0.0.1:6000")

Wrapper tries to connect to server <tcp://127.0.0.1:6000>
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


```setupwithmeshcat``` takes an optional parameter corresponding to the url (zmq_url) of the meshcat server (given as a string). If no url is provided, it uses the variable ```MESHCAT_URL``` defined in config.py . If for some reason the default url does not match the one you are using, you can either provide this url to ```setupwithmeshcat``` or simply replace the value of ```MESHCAT_URL```

In [23]:
from config import MESHCAT_URL
print(MESHCAT_URL)

tcp://127.0.0.1:6000


In [24]:
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()

## Description of the environment.
The environment is composed of a fixed table and an obstacle, as well as a cube, which you are supposed to bring 
to the green target. If you run ```inverse_geometry.py``` you should automatically load the environment and the robot.

## Description of the robot. 
The robot you will use for the lab is the Nextage robot from Kawada industries.
You can use both pinocchio and the [URDF files](https://github.com/ediadvancedrobotics/lab/blob/1d00a1c79acbcb7248df1b17cf3aaac7f8c39a8b/models/nextagea_description/urdf/NextageaOpen.urdf) to check the dimension of the robot configuration space.
The configuration velocity space has the same dimension as Nextage is only composed of revolute joints.

In [25]:
print(robot.model)

Nb joints = 16 (nq=15,nv=15)
  Joint 0 universe: parent=0
  Joint 1 CHEST_JOINT0: parent=0
  Joint 2 HEAD_JOINT0: parent=1
  Joint 3 HEAD_JOINT1: parent=2
  Joint 4 LARM_JOINT0: parent=1
  Joint 5 LARM_JOINT1: parent=4
  Joint 6 LARM_JOINT2: parent=5
  Joint 7 LARM_JOINT3: parent=6
  Joint 8 LARM_JOINT4: parent=7
  Joint 9 LARM_JOINT5: parent=8
  Joint 10 RARM_JOINT0: parent=1
  Joint 11 RARM_JOINT1: parent=10
  Joint 12 RARM_JOINT2: parent=11
  Joint 13 RARM_JOINT3: parent=12
  Joint 14 RARM_JOINT4: parent=13
  Joint 15 RARM_JOINT5: parent=14



You can also verify that in its default configuration the robot is in collision (with the table)

In [26]:
from tools import collision
import numpy as np
collision(robot, robot.q0)

True

# Configuration and helper functions  <a class="anchor" id="config"></a>

I have modified the URDF files to add frames that are relevant for the tasks you need to accomplish.
On the robot, I have created fixed joints attached to the tip of each effectors, called ```LARM_EFF``` and ```RARM_EFF```. Note that because they are fixed joints, they do not appear in the robot model and do not change the dimension of the configuration space. These names are defined for you in the config.py file as well. 

In [27]:
from config import LEFT_HAND, RIGHT_HAND

print ("Left hand joint name: ", LEFT_HAND)


import pinocchio as pin
q = robot.q0.copy()

#update the frame positions in robot.data given q
pin.framesForwardKinematics(robot.model,robot.data,q)

#now let's print the placement attached to the right hand
print ("Left hand joint placement: ")
pin.computeJointJacobians(robot.model,robot.data,q)
frameid = robot.model.getFrameId(LEFT_HAND)
oMframe = robot.data.oMf[frameid] 
print(oMframe)



Left hand joint name:  LARM_EFF
Left hand joint placement: 
  R =
-3.67321e-06           -1            0
           1 -3.67321e-06            0
           0            0            1
  p = 0.452  0.28 0.851



Likewise the cube urdf (models/cubes/cube_small.urdf) also contains helpers joint that set a target location for the effectors. They are called ```LEFT_HOOK``` and ```RIGHT_HOOK``` in config.py.

I added meshcat helper functions that will allow you to display the associated frames if you wish.

In [28]:
from config import LEFT_HOOK, RIGHT_HOOK, CUBE_PLACEMENT, CUBE_PLACEMENT_TARGET

from tools import getcubeplacement, setcubeplacement
from setup_meshcat import updatevisuals

#We can access the current cube position using
oMcube  = getcubeplacement(cube) #origin of the cube
oMcubeL = getcubeplacement(cube, LEFT_HOOK) #placement of the left hand hook
oMcubeR = getcubeplacement(cube, RIGHT_HOOK) #placement of the right hand hook


#the cube position is updated using the following function:
setcubeplacement(robot, cube, CUBE_PLACEMENT)
#to update the frames for both the robot and the cube you can call
updatevisuals(viz, robot, cube, q)


In [29]:
frameid = robot.model.getFrameId('LARM_EFF')
oMframe = robot.data.oMf[frameid]
oMframe

  R =
-3.67321e-06           -1            0
           1 -3.67321e-06            0
           0            0            1
  p = 0.452  0.28 0.851

In [30]:
frameid = robot.model.getFrameId('RARM_EFF')
oMframe = robot.data.oMf[frameid]
oMframe

  R =
-3.67321e-06           -1            0
           1 -3.67321e-06            0
           0            0            1
  p = 0.452 -0.28 0.851

# Part 1: Geometry <a class="anchor" id="part1"></a>
In this first part of the lab, we are concerned with the geometry of the robot and motion planning in general.
We are going to compute a geometric path that will serve as a guide for the dynamic part. Concretely, your objective is to compute a collision free path for the robot such that it "grasps" the cube and carries it over to its target position. Again, no dynamic computations are required in this phase, we are only interested in configurations that should be collision free, respect joint limits and geometrically consistent in terms of placement; there is already some work to be done here!

## I. Computing the target configurations (Inverse Geometry) <a class="anchor" id="IG"></a>

Your first task is the following: write the functions that generate an initial and a goal configuration for the nextage robot, such that the ```LEFT_HAND``` and ```RIGHT_HAND```are aligned respectively with the ```LEFT_HOOK``` and ```RIGHT_HOOK``` frames when the cube is located at its starting position  (```CUBE_PLACEMENT```) and at its goal position (```CUBE_PLACEMENT_TARGET```)

For this implement the method ```computeqgrasppose``` in inverse_geometry.py 

The main method indicates how the method is going to be called to obtain the q0 and qe configurations:
```
q0 = computeqgrasppose(robot, q, cube, CUBE_PLACEMENT, viz)
qe = computeqgrasppose(robot, q, cube, CUBE_PLACEMENT_TARGET,  viz)
```

Of course, q0 and qe should be collision-free and respect the joint limits of the robot. Do not hard-code anything here. In my tests I will use different targets to test the generality of ```computeqgrasppose```.

q0 should look somehow like this:
<img src="./images/q0.png" alt="drawing" width="200"/>



**hints:** 
* If your configurations look unnatural, you probably want to somehow introduce a "postural bias" in whatever method you are using 
* From the obtained configurations, you can easily obtain the relative placement between both hands expressed in a specific frame. This might prove relevant later in the lab so you may want to store it somewhere (maybe in solution.py)


In [31]:
from inverse_geometry import computeqgrasppose

In [32]:
q0, success = computeqgrasppose(robot, q, cube, CUBE_PLACEMENT, viz)
if success:
    updatevisuals(viz, robot, cube, q=q0)

In [33]:
qe, success = computeqgrasppose(robot, q, cube, CUBE_PLACEMENT_TARGET,  viz)
if success:
    updatevisuals(viz, robot, cube, q=qe)

## II. Motion planning <a class="anchor" id="motion_planning"></a>

For section II, I am proposing one course of action to successfully achieve the planning of a motion such that the robot carries the cube from its initial configuration to the end configuration. You are free to choose another course of action, but I do not advise it unless you really know what you are doing. In any case talk to the TAs or myself before making that decision. No matter what approach you choose, the method ```computepath``` from path.py must be implemented as specified for you to get the points.

To compute a geometric path that represents a collision-free motion of our robot carrying the cube from q0 to qe, I suggest to use a sampling based motion planner. 

We will use the path.py file to write the motion planning algorithm

### II.a Sampling configurations
To generate configurations for the planner, I suggest a 2 step process: randomly sample configurations for the cube, then solve an inverse geometry problem to generate a valid pose on that location. Of course, check that the joint limits are respected and the configuration is collision-free before returning it.

**hint:** you are free to bound your problem by ensuring that the cube placements that you generate only occur at positions / orientations that you think are interesting.

### II.b Path projection
However to do this we need to enforce the constraint that every sampled configuration is such that the effectors are holding the cube.

Furthermore, the grasping constraint will apply to the complete path: a standard linear interpolation between two grasping configurations is not enough to guarantee that every interpolated configuration is such that the cube is grasped.
Write a function that, given two configurations q0 and q1 and a discretisation step, returns an interpolated path between q0 and q1 such that every configuration in the path corresponds to a grasping pose. If it is not possible to generate such path, it will return a flag indicating so and, depending on your own decision, either return the part of the path that is valid, or nothing

### II.c Solution path generation
With the methods produced in II.1 and II.2, you should now be able to implement a motion planner that generates a geometrically valid path between q0 and qe. You can probably reuse code from the motion planning tutorial here, or implement you own. 

**requirement:** For me to assess this part you are required to implement the method ```computepath``` according to its documentation

In [34]:
from path import *

In [35]:
graph, results = create_path(
    robot,
    cube, 
    n_samples=500,
    q0=q0,
    c0=CUBE_PLACEMENT,
    qe=qe,
    ce=CUBE_PLACEMENT_TARGET,
    control=True,
    n_steps_interpol=30,
    n_steps_graph_interpolations=5
)
results

Creating sample path: 


  9%|#########                                                                                           | 45/500 [00:05<00:20, 22.08samples/s]

Original target right [0.41034508 0.22974914 1.13929663]
Original target right [0.41034508 0.32974914 1.13929663]
End target right [0.41034508 0.23074914 1.13929663]
End target right [0.41034508 0.32874914 1.13929663]


 12%|############2                                                                                       | 61/500 [00:06<00:26, 16.69samples/s]

Original target right [ 0.24105886 -0.47398338  0.94392536]
Original target right [ 0.24105886 -0.37398338  0.94392536]
End target right [ 0.24105886 -0.47298338  0.94392536]
End target right [ 0.24105886 -0.37498338  0.94392536]
Original target right [ 0.3270353  -0.35413278  0.93046418]
Original target right [ 0.3270353  -0.25413278  0.93046418]
End target right [ 0.3270353  -0.35313278  0.93046418]
End target right [ 0.3270353  -0.25513278  0.93046418]
Original target right [ 0.32407059 -0.35826556  0.93092836]
Original target right [ 0.32407059 -0.25826556  0.93092836]
End target right [ 0.32407059 -0.35726556  0.93092836]
End target right [ 0.32407059 -0.25926556  0.93092836]
Original target right [ 0.32110589 -0.36239834  0.93139254]
Original target right [ 0.32110589 -0.26239834  0.93139254]
End target right [ 0.32110589 -0.36139834  0.93139254]
End target right [ 0.32110589 -0.26339834  0.93139254]
Original target right [ 0.31814118 -0.36653112  0.93185671]
Original target righ

...

Original target right [ 0.43408257 -0.46412387  1.27728309]
Original target right [ 0.43408257 -0.36412387  1.27728309]
End target right [ 0.43408257 -0.46312387  1.27728309]
End target right [ 0.43408257 -0.36512387  1.27728309]
Original target right [ 0.51460687 -0.39040794  1.23391289]
Original target right [ 0.51460687 -0.29040794  1.23391289]
End target right [ 0.51460687 -0.38940794  1.23391289]
End target right [ 0.51460687 -0.29140794  1.23391289]
Original target right [ 0.51183017 -0.39294986  1.23540841]
Original target right [ 0.51183017 -0.29294986  1.23540841]
End target right [ 0.51183017 -0.39194986  1.23540841]
End target right [ 0.51183017 -0.29394986  1.23540841]
Original target right [ 0.50905347 -0.39549179  1.23690393]
Original target right [ 0.50905347 -0.29549179  1.23690393]
End target right [ 0.50905347 -0.39449179  1.23690393]
End target right [ 0.50905347 -0.29649179  1.23690393]
Original target right [ 0.50627677 -0.39803372  1.23839946]
Original target righ

 95%|##############################################################################################4    | 477/500 [00:56<00:04,  5.54samples/s]

Original target right [ 0.41249694 -0.13217875  1.05733713]
Original target right [ 0.41249694 -0.03217875  1.05733713]
End target right [ 0.41249694 -0.13117875  1.05733713]
End target right [ 0.41249694 -0.03317875  1.05733713]


100%|###################################################################################################| 500/500 [00:57<00:00,  8.72samples/s]


True

In [36]:
len(graph.nodes)

120

In [37]:
path, length = shortest_path(graph)

q_path = [graph.nodes[id].q for id in path]

In [38]:
displaypath(robot,q_path,dt=0.5,viz=viz)

In [39]:
import pickle as pkl
pkl.dump(q_path, open("path_control_2.pkl", "wb"))

In [183]:
path, length = shortest_path(graph)
length

0.6955278310644804

In [184]:
len(graph.nodes)

152

In [185]:
path

[0, 2, 27, 28, 29, 30, 1]

In [33]:
N=500
q0=q0
c0=CUBE_PLACEMENT
qe=qe
ce=CUBE_PLACEMENT_TARGET
n_steps_interpol=15


In [37]:
g.nodes

[<__main__.Node at 0x7f939c7db100>, <__main__.Node at 0x7f939c7d8370>]

In [35]:
start_node = Node(c0, q0)
g = Node_Graph(start_node)
end_node = Node(ce, qe)
end_node_id = g.add_node(end_node)

prev_q = q0.copy()
available_path = False
print("Creating sample path: ")


Creating sample path: 


In [38]:
# maybe make a better randomizer how this is done 
sampled_cube = generate_random_cube()

In [39]:


qt, success_grasp = computeqgrasppose(robot, q0, cube, sampled_cube, viz=None)
if success_grasp:
    closest_node_id = g.find_closest_neigbour(sampled_cube)
    closest_node = g.nodes[closest_node_id]

    qs_interpolated = compute_interpolation(robot, cube, closest_node.q, closest_node.cube, sampled_cube, n_steps=n_steps_interpol)

    if len(qs_interpolated) > 1:
        last_node_id = add_interpolations(g, qs_interpolated, closest_node_id, n_interpolations=2)

        qs_interpolated_end = compute_interpolation(robot, cube, qs_interpolated[-1][1], qs_interpolated[-1][0], ce, n_steps=n_steps_interpol)
        if len(qs_interpolated_end) > 1:

            add_interpolations(g, qs_interpolated_end, last_node_id, n_interpolations=2)

            if len(qs_interpolated_end) == n_steps_interpol+1:
                available_path = True

In [41]:
last_node_id

NameError: name 'last_node_id' is not defined

In [18]:
path = computepath(q0,qe,CUBE_PLACEMENT, CUBE_PLACEMENT_TARGET)


TypeError: computepath() missing 2 required positional arguments: 'cubeplacementq0' and 'cubeplacementqgoal'

In [ ]:
path = computepath(q0,qe,CUBE_PLACEMENT, CUBE_PLACEMENT_TARGET)

displaypath(robot,path,dt=0.5,viz=viz) #you ll probably want to lower dt


In [104]:
path, length = shortest_path(graph)
path

NameError: name 'graph' is not defined

# Part 2: Dynamics <a class="anchor" id="part2"></a>
Now that you have computed a reference path for your robot, it is time to try controlling it in a dynamics simulator. We will use pybullet for this. In theory pybullet is already installed but if not you should install it with pip:





In [1]:
!pip install pybullet

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip



This second part consists in two tasks. First, converting your path into a time-parametrised trajectory, then tracking this trajectory using the control law of your choice.

Again, you are free to use any method that you want to both tasks. 

## I. From a path to a trajectory <a class="anchor" id="TO"></a>
The first step is to parametrise your path into a time-varying trajectory that you may want to track.
Depending on your choice of a control law, this trajectory could be in the configuration space, the task space, or even something else.

I do not provide a template for this step, you are free to use any method that you see fit, so if you don't want to be bothered you can simply manually parametrise your trajectory (which is the level 0 described thereafter). However, you can score bonus marks if you go with more advanced methods. This will be described in the coming marking criteria. 


### 1. level 0: Manually set a velocity profile
Come up with a time parametrisation for your path that intuitively makes sense and move on to the control part! To do this write an interpolation function that given a certain $t$ will return a configuration in your path as well as the corresponding velocity and accelerations desired at that time. I advise to work out something that is such that the starting and final velocities are 0. 

### 2. level 1: QP programming without collision constraints (covered in next week's lecture)
What I would recommend is to use quadratic programming to solve a least square problem that fits at best the path that you computed while satisfying constraints.

The class Bezier defined in [bezier.py](https://github.com/ediadvancedrobotics/lab/blob/main/bezier.py) might prove useful.
Take a look at the [primer on Bezier curves](https://pomax.github.io/bezierinfo/) if you want more info on these methods. An example of use is provided in [control.py](https://github.com/ediadvancedrobotics/lab/blob/1d00a1c79acbcb7248df1b17cf3aaac7f8c39a8b/control.py#L47).

The motivations for using Bezier curves are multiple:

+ First, they are strictly equivalent to polynomials, which means that the trajectory you will compute will be continuous and infinitely differentiable.

+ Secondly, the initial and terminal conditions are easy to specify: the velocity at the start / end only depends on the first/last two control points, and the accelerations only on the the first/last three control points. If you choose your first three control points to be strictly equal for instance, the initial velocity and acceleration of your trajectory will be 0.

+ Thirdly, a Bezier curve lies completely in the convex hull of its control points. What this means for optimisation is that if you define linear inequality constraints on the control points of the curve, you have the guarantee that that every point on the curve satisfies these constraints. This will allow you to easily specify constraints on the derivatives

If you choose to go that way, you can decide that your optimisation variables are the control points of your Bezier curve; you can then linearly define the value of the trajectory at each time step as a linear combination of these control points and write your cost function as a function of these points.


In this approach, the idea is not to explicitely address the collision constraints. You will assume that if you track your path well enough you will avoid collisions. This is commonly done in robotics.

**Optional: the ndcurves library**

If you are interested in trajectory optimisation, you could check out [this tutorial I wrote for the ndcurves library](https://github.com/loco-3d/ndcurves/blob/master/python/test/sandbox/test.ipynb). While the API is different the concepts described are the same. **Note that I m not necessarily suggesting to use ndcurves, simply to look at the tutorials to understand the ideas and replicate this.** Still, if you want to you can decide to directly use ndcurves. It has an api a bit more complex that the simple Bezier class I provided as it is more powerful, so it is for you to decide, there is no good or bad solution here.



#### Time parametrisation:
If you follow the guidelines from the tutorial, the optimisation will give you a smooth trajectory, but by default it will have a duration of 1. You can easily add velocity constraints and include time as a variable in your problem . You could also add acceleration constraints and ignore velocities by using $t^2$ as a variable instead of $t$. We can discuss this on Piazza. You can't consider both velocity and acceleration constraints otherwise your problem will become non-linear (do you see why ?)

#### Handling grasping constraints:
If you are planning a trajectory just for the effectors then it is really easy to handle these constraints: you can just plan for the cube and then deduce the effector placements at each step.
It might be harder to do it for a configuration space trajectory, but it is not necessarily required:
I believe that if the tracked trajectory is good enough you should not need to handle the grasping constraints as accurately as this will be fixed by the control law. 
If you want to handle these constraints, there are a variety of post processes that you can choose to implement this, but though the problem becomes non-linear again... I suggest to avoid this unless it really proves useful (again I think not, or at least it was not necessary for me).



### 3. level 2: Handling collision constraints
First of all, don't go there in the first instance. Wait until you are done with the complete lab to decide whether you are interested in doing this. You can try to handle collisions in a variety of ways that we can discuss on Piazza. The straightforward approach is to write a non linear program that will handle this as we have done before. Because you have a reference motion that is collision free, you may get away with a NLP to refine the trajectory obtained when solving level 1 so as to avoid collisions.

## II. Implementing a torque control law <a class="anchor" id="control"></a>
Ok now, you have everything that you need to grasp that box and move it to its target.

Again, we are going to use pybullet to control our robot. Pybullet is a dynamics simulator which is used by roboticists to test the behaviours of their controllers before deploying them on their robots. It was originally developed for video games, and depending on your application it is not always considered as very accurate. However for manipulators it works well.

### Interaction with Pybullet 

The philosophy I propose to interact with Pybullet is implemented in the [control.py main function](https://github.com/ediadvancedrobotics/lab/blob/1d00a1c79acbcb7248df1b17cf3aaac7f8c39a8b/control.py#L61):

+ Set the initial configuration of your robot in pybullet (method setqsim)

+ At each frame:

    - get the current state of the robot in the simulator (method getpybulletstate)
    
    - work out all the computations in pinocchio
    
    - send a torque command to the simulator to update the state of the robot (method step)

Pybullet is configured here to work as a step-by-step simulation, as opposed to real time. Everytime you call the step method, the simulation integrates DT seconds of time, with DT defined in config.py.

In tools.py, I have added a method [rununtil](https://github.com/ediadvancedrobotics/lab/blob/1d00a1c79acbcb7248df1b17cf3aaac7f8c39a8b/tools.py#L102) that you can use to update the simulation at a frequency that corresponds to a realtime mode.

If you run control.py, assuming the initial state is the robot.q0 configuration, the first thing you will see is a very chaotic behaviour from the robot: indeed, as it starts in collision, the simulator will apply really high forces to compense the penetration violation.

If you initialise correctly the inital state, in the absence of torque command the robot arms will fall under the effect of gravity.

The main methods needed to work with pybullet are defined in the Simulation class defined in [setup_pybullet.py](https://github.com/ediadvancedrobotics/lab/blob/1d00a1c79acbcb7248df1b17cf3aaac7f8c39a8b/setup_pybullet.py). A few others are defined in the base class Simulation_base but I don't expect you will need them.

The helper method to load the robot in both pinocchio and pybullet environments is setupwithpybullet. If you still want to work with meshcast in parallel (for some strange reason that I can't explain I found it convenient) you can instead call setupwithpybulletandmeshcat.


### Optional task 0: Control without the cube
To test your controller, it might be a good idea to first control the motion of the effectors without grasping the cube. Generate a simple trajectory that brings the effectors above the starting position and implement the control law to achieve the motion. Regarding the gains, you definitely have the options to tune them individually for each joint (and you would have do this on the actual robot). When I programmed the lab I just used $K_p = 300$ and $K_d = 2 \sqrt(K_p)$ for all joints and it worked fine. Once you ll have verified that you obtain a satisfying behaviour, you'll be ready to move to the actual task.

### The actual task
Proceed as you wish to have the robot grasp the cube and then bring it to the target location. It does not matter to me how this is achieved in terms of the motion I see, as long as some grasping (ie the robot is holding the cube above the table for some relevant period of time) happens. You can push the cube to align it in the end if you need. 

To achieve the behaviour, you will need to apply a control law. Everything here is set for you to control the robot in torque, which I do believe is the easy way to go. If you want to try controling this using position or velocity control, this is also an option (actually the real Nextage robot is only position controlled).

The recommended way, in my opinion, is to apply inverse dynamics control to track the trajectory while applying a linear force to the cube with both hands.

This force control law should naturally cope with the alignment errors and have the effect of attracting the effectors to the cube 

